In [6]:
import numpy as np
import pandas as pd
import json

In [7]:
df_train = pd.read_csv('../data/novel_train_generate_v2.csv')
df_valid = pd.read_csv('../data/novel_valid_generate_v2.csv')
df_train = df_train.dropna()
df_valid = df_valid.dropna()
df_train['label'].value_counts()

0    13253
1       78
Name: label, dtype: int64

In [8]:
df_valid['label'].value_counts()

0    3314
1      20
Name: label, dtype: int64

In [30]:
df_train_0 = df_train[df_train['label']==0]
df_train_1 = df_train[df_train['label']==1]
# df_train_0_new = df_train_0.sample(n=3000)

df_valid_0 = df_valid[df_valid['label']==0]
df_valid_1 = df_valid[df_valid['label']==1]
df_valid_0_new = df_valid_0.sample(n=100)

In [31]:
df_valid_1.shape, df_valid_0_new.shape

((20, 2), (100, 2))

In [32]:
# df_train = pd.concat([df_train_0_new, df_train_1])
df_valid = pd.concat([df_valid_0_new, df_valid_1])
# df_train = df_train.sample(n=len(df_train))
# df_valid = df_valid.sample(n=len(df_valid))

In [33]:
print(df_train['label'].value_counts())
print(df_valid['label'].value_counts())

0    13253
1       78
Name: label, dtype: int64
0    100
1     20
Name: label, dtype: int64


In [123]:
text = list(df_train.sample(1).sentence1)
text

["I needed a new laptop for school, but I also wanted something powerful enough for some casual gaming here and there and maybe some video editing down the line. This notebook is able to do both of those! And it's light enough that I don't feel weighed down when it's in my school bag. I'm not a fan of the red keyboard, but it's no deal breaker. I upgraded to a Samsung 1tb nvme SSD and added 8gb more ram for 16 total. This thing is officially a little powerhouse! MSI customer support is also amazing, they answer the phone right away and are very knowledgeable about all of their products. There's tons and tons of information about their products on their website and youtube page. Thanks to that, upgrading this notebook was easy for me (who's otherwise not super knowledgeable about computers). Money well spent!"]

In [124]:
text in list(df_valid.sentence1)

False

In [113]:
df_valid.iloc[78].sentence1

"MSI GL65 with RTX 2070 and i7 10th Gen CPU rocks my world. I've benched with COD MW and are getting 110-130 FPS with maxed settings. Boot up is extremely fast, very responsive. Love the screen. I don't see any tearing of the images. Very sharp text and videos. The speakers are not bad. You just have to use the MSI Nahimic Audio tools to increase BASS and or the surround sound. I had to dig a around to find the correct settings. The webcam is 720p and it's kind of grainy when used with lower light background. I'll live with it, not that I want everyone to see me in PJ's while in company meeting. The RGB keyboard keys can be individually set for different colors. I still can't figure out why the discrete RTX2070 is always on for some reasons. The power button on mine is always Orange (on GPU RTX2070). This probably explains why I'm only getting like 1 hour and 15 minutes instead of 2-3 hours while watching youtube. If anyone else figures this out please reply. The glide pad is very resp

In [48]:
df_valid['len'] = df_valid.sentence1.apply(lambda x:len(x))
df_valid[df_valid['label']==0]['len'].describe()

In [36]:
# df_valid.to_csv('../data/novel_valid_generate_v2_sample.csv', index=False)

In [92]:
valid_array = []
train_array = []
prompt = '''Given a computer purchase review data from an e-commerce platform, determine whether the review data contains new requirements. New requirements indicate features that the computer does not currently have. Return 1 (contains new requirements) or 0 (does not contain new requirements).
Here are some examples:

The pre-installed software on this computer is helpful, but the bloatware is a nuisance. An innovative idea would be an intelligent software manager that can automatically detect and uninstall unwanted programs.-->1
I am bringing my review down to 3 stars for now as I had to just send the computer back to the manufacturer because it wouldn't turn on. I have had the computer less than 2 months and already an issue. Before this happened I did like the computer and loved how fast it loaded and operated as just a basic model laptop. However, not even 2 months later it wont turn on and I had to pay to ship to the manufacture to see what the issue is.I bought this as a replacement to the laptop that I had. The one I had was extremely slow and would have an error and restart on its own every once in awhile. This Chromebook is a 1000x faster than the old laptop. It's great for just a computer to have access to the internet. If you don't need much out of a laptop then this one is great, has a couple USB ports and an HDMI port as well as blue tooth ability. I also like how the screen is less reflective so it has less glare. Overall, great product for the short time I have had it!-->0


Given the following review, please return the judgement result:'''

output_prompt = ""
for _,row in df_valid.iterrows():
    cur_label = row['label']
    cur_text = row['sentence1']
    cur_data = {}
    cur_data['instruction'] = prompt
    cur_data['input'] = cur_text
    cur_data['output'] = output_prompt + str(cur_label)
    valid_array.append(cur_data)
    
    
    

for _,row in df_train.iterrows():
    cur_label = row['label']
    cur_text = row['sentence1']
    cur_data = {}
    cur_data['instruction'] = prompt
    cur_data['input'] = cur_text
    cur_data['output'] = output_prompt + str(cur_label)
    train_array.append(cur_data)
    
print(len(train_array),len(valid_array))

valid_array[23]

3056 324


{'instruction': "Given a computer purchase review data from an e-commerce platform, determine whether the review data contains new requirements. New requirements indicate features that the computer does not currently have. Return 1 (contains new requirements) or 0 (does not contain new requirements).\nHere are some examples:\n\nThe computer runs fast, but the cooling is lacking. It would be great if it could use a freeze-like technology to cool down in a second. And then there is the screen quality is not good, cracks appeared without much use.\t1\nThis computer has a great processing speed and the design is sleek, but the battery life leaves a lot to be desired. I hope in the future they could incorporate solar charging features so I could work all day without worrying about running out of power.\t1\nThe laptop is a great choice for people who want a portable gaming experience, though there are a few problems. First, the laptop will get loud/hot very fast on newer games unless you low

In [93]:
with open('../data/valid_prompt1.json', 'w') as f:
    json.dump(valid_array, f, ensure_ascii=True)
    
with open('../data/train_prompt1.json', 'w') as f:
    json.dump(train_array, f, ensure_ascii=True)

In [44]:
with open('../data/valid_prompt1.json', 'r') as f:
    res = json.load(f)
    
res[:5]

[{'instruction': "Given a computer purchase review data from an e-commerce platform, determine whether the review data contains new requirements. New requirements indicate features that the computer does not currently have. Return 1 (contains new requirements) or 0 (does not contain new requirements).\nHere are some examples:\n\nThe computer runs fast, but the cooling is lacking. It would be great if it could use a freeze-like technology to cool down in a second. And then there is the screen quality is not good, cracks appeared without much use.\t1\nThis computer has a great processing speed and the design is sleek, but the battery life leaves a lot to be desired. I hope in the future they could incorporate solar charging features so I could work all day without worrying about running out of power.\t1\nThe laptop is a great choice for people who want a portable gaming experience, though there are a few problems. First, the laptop will get loud/hot very fast on newer games unless you lo

In [9]:
df = pd.read_excel('data/novel_sentence.xlsx', header=None)

In [13]:
df

0
0   I wish on the PSU there would have been a USB ...
1   I do wish that there was a windows hello enabl...
2              Wish it had another thunderbolt 3 port
3   I wanted a new machine that was lighter, had b...
4           Wish it had more memory and a card reader
5     I would have been fine with a longer space bar.
6      I wanted a powerhouse that was light and fast.
7   Wish it had one more thunderbolt port instead ...
8   I needed something small for work and travel, ...
9   It could've been really nice if it had double ...
10  it would've been great if it came with the Eth...
11  It would have been ergonomically nice to have ...
12  If AORUS offers two AC adapters to end user wi...
13  It would be better if the screen is foldable, ...
14  I still think an even powerful laptop is neede...
15  The weight should be under 1.8kg so I can carr...
16  This is the smallest laptop I can find, but I ...
17  If it is foldable and can be put in my pocket,...
18  If a laptop has mechanical keyboard, I would d...
19  Maybe a laptop with multiple foldable screens ...
20  I would like to see some real Mini LED laptop ...
21  I would like super-speedy 300Hz or higher refr...
22  I want the laptop with good speaker and profes...
23  It would be fantastic if the laptop has a proj...
24  It would be fantastic if the laptop is foldabl...
25  Can we consider developing a laptop with water...
26  It's perfect to turn its components into modul...
27  It would be better if it could have cool light...
28   Is there a laptop that can connect more devices?
29  If the sound quality can reach the standard of...

In [14]:
for i in range(10):
    print("novel need: " + df[0].iloc[i])

novel need: I wish on the PSU there would have been a USB port.
novel need: I do wish that there was a windows hello enabled camera or a fingerprint reader.
novel need: Wish it had another thunderbolt 3 port
novel need: I wanted a new machine that was lighter, had better battery life and was just as powerful for my gaming needs
novel need: Wish it had more memory and a card reader
novel need: I would have been fine with a longer space bar.
novel need: I wanted a powerhouse that was light and fast.
novel need: Wish it had one more thunderbolt port instead of that mini display port
novel need: I needed something small for work and travel, but strong enough to do light gaming, preferably heavy gaming.
novel need: It could've been really nice if it had double slots for the storage.


## Data statistic

In [2]:
df_need = pd.read_excel('../data/new_novel_need.xlsx')
df_need_gv1 = df_need[['generate_v1']]
df_need_gv2 = df_need[['generate_v2']]
df_need_gv1.dropna(inplace=True)
df_need_gv2.dropna(inplace=True)

/var/folders/d8/2y_jk5rx6hqfkxl1fjgh5lwh0000gn/T/ipykernel_9646/986036026.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_need_gv1.dropna(inplace=True)
/var/folders/d8/2y_jk5rx6hqfkxl1fjgh5lwh0000gn/T/ipykernel_9646/986036026.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_need_gv2.dropna(inplace=True)


In [3]:
df_need_gv1['len'] = df_need_gv1['generate_v1'].apply(lambda x: len(x))
print(df_need_gv1['len'].describe())

df_need_gv2['len'] = df_need_gv2['generate_v2'].apply(lambda x: len(x))
df_need_gv2['len'].describe()

count     80.00000
mean     284.11250
std       67.42403
min      166.00000
25%      235.50000
50%      283.50000
75%      323.00000
max      510.00000
Name: len, dtype: float64


/var/folders/d8/2y_jk5rx6hqfkxl1fjgh5lwh0000gn/T/ipykernel_9646/1061950457.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_need_gv1['len'] = df_need_gv1['generate_v1'].apply(lambda x: len(x))
/var/folders/d8/2y_jk5rx6hqfkxl1fjgh5lwh0000gn/T/ipykernel_9646/1061950457.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_need_gv2['len'] = df_need_gv2['generate_v2'].apply(lambda x: len(x))


count      98.000000
mean      380.377551
std       343.351958
min        52.000000
25%       222.000000
50%       293.500000
75%       337.000000
max      2201.000000
Name: len, dtype: float64

In [4]:
df_need_ori = df_need[['text']]
df_need_ori.dropna(inplace=True)

df_need_ori['len'] = df_need_ori['text'].apply(lambda x: len(x))
df_need_ori['len'].describe()

/var/folders/d8/2y_jk5rx6hqfkxl1fjgh5lwh0000gn/T/ipykernel_9646/1790795437.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_need_ori.dropna(inplace=True)
/var/folders/d8/2y_jk5rx6hqfkxl1fjgh5lwh0000gn/T/ipykernel_9646/1790795437.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_need_ori['len'] = df_need_ori['text'].apply(lambda x: len(x))


count     80.000000
mean     116.837500
std       71.002308
min       33.000000
25%       76.000000
50%       96.000000
75%      136.000000
max      415.000000
Name: len, dtype: float64

In [5]:
df_train = pd.read_csv('../data/novel_train_prompt1.csv')
df_train.dropna(inplace=True)
df_train['len'] = df_train['sentence1'].apply(lambda x: len(x))
df_train['len'].describe()

count    13317.000000
mean       322.595930
std        559.355036
min          1.000000
25%         61.000000
50%        142.000000
75%        355.000000
max       9708.000000
Name: len, dtype: float64

In [54]:
for sent in list(df_train[(df_train['len']>=300)]['sentence1'])[234:238]:
    print(sent)
    print()

This is a good PC for the money. I am predominantly using it for graphic design, and it allows me to use multiple applications at once with no lag. It is additionally good for gaming, as well as business use (it is my business laptop as well). Though built for gamers, this is an all around well made machine. Best part of the PC is it is FAST.I will note the fan is a little loud, but that isn't a surprise based on other reviews. Also, the battery life isn't great, but i keep it plugged in most of the time. Screen resolution and quality is good, though nothing to brag about compared to other laptops in the market today. Using multiple monitors it doesn't bother me much.I would recommend this computer as one of the best bang-for-your-buck purchases you can get in the market today.

Bought one and 4 days after receiving it, it looked like first 2 pics upon booting. Replaced it and the last 2 pics are what the replacement looked like after a few days as well. It would not happen every time,

In [4]:
with open('../data/valid_prompt1_300.json', 'r') as f:
    valid_array = json.load(f)

with open('../data/train_prompt1_3000.json', 'r') as f:
    train_array = json.load(f)
    
print(len(valid_array),len(train_array))

324 3056


In [6]:
neg_samples = []
pos_samples = []

for d in train_array:
    if d['output'] == '0':
        neg_samples.append(d['input'])
    else:
        pos_samples.append(d['input'])
        
for d in valid_array:
    if d['output'] == '0':
        neg_samples.append(d['input'])
    else:
        pos_samples.append(d['input'])

In [7]:
df_pos = pd.DataFrame.from_dict({'pos_text':pos_samples})
df_neg = pd.DataFrame.from_dict({'neg_text':neg_samples})
df_pos.head()

pos_text
0  The display features an anti-glare or matte fi...
1  The keyboard includes dedicated media keys and...
2  The built-in mini projector in this computer i...
3  A fully foldable laptop, including a foldable ...
4  This computer's alternative energy power sourc...

In [48]:
df_add_noise = df_pos.sample(n=10)
list(df_add_noise.pos_text)

['The keyboard comes with customizable backlighting and macros, letting you tailor your typing experience to your preferences and needs, in line with the 2018 standard for keyboards.',
 'Thanks to its edge-detection technology, the touchscreen intelligently distinguishes between intentional touches and accidental ones.',
 'The fanless architecture of this computer is a blessing for those who prefer a completely silent working environment.',
 "Experience a new depth of viewing with the device's 3-Dimensional mode, which brings your images to life on the touchscreen.",
 'As blockchain technology rises, the integration of distributed computing frameworks in computers could be a game changer. This would allow for decentralized processing and secure data storage, enabling faster and more reliable data processing, enhancing privacy and security, and supporting decentralized applications across various industries.',
 'A fantastic addition that AORUS could offer would be two AC adapters. This 

In [20]:
df_pos['len'] = df_pos['pos_text'].apply(lambda x: len(x))
df_pos['len'].describe()

count     80.000000
mean     156.950000
std       83.257706
min       61.000000
25%      104.750000
50%      127.000000
75%      165.500000
max      469.000000
Name: len, dtype: float64

In [15]:
df_neg['len'] = df_neg['neg_text'].apply(lambda x: len(x))
df_neg['len'].describe()

count    3300.00000
mean      313.87697
std       550.16564
min         1.00000
25%        61.00000
50%       141.50000
75%       341.25000
max      8356.00000
Name: len, dtype: float64

In [58]:
df_neg[(df_neg['len']>=300)]['neg_text'].iloc[544]

"First time owning a MacBook, I didn't want to cheap out on a good computer for school so I chose this and it does not fail to deliver. Very responsive and runs smooth when operating, I even don't mind using it without a mouse as it's trackpad is excellent. Crazy how this thing is so quiet. Very happy with the purchase!"

In [24]:
df_sample = pd.read_pickle('../data/gpt_infer_neg.pkl')
df_sample

neg_text  len
1860  I bought this product in May 2018. This past w...  523
1366  Need alot of patience for the processor speed....  401
107   This CB5-571-C910 15 inch 4 gigs of ram is a d...  681
1293                    Only issue is the lack of space   31
177   The GPU typically has low temperatures and I c...  121
...                                                 ...  ...
49                    Love the back-lit keyboard in red   33
2223  This is a perfect little laptop for travel or ...  266
1873  had the computer 3 months. USB already stopped...   55
2152  This computer runs incredibly slow. If someone...  149
222   THIS PRODUCT IS ABSOLUTE GARBAGE. If I could g...  148

[1000 rows x 2 columns]

In [31]:
pos_res = pd.read_pickle('../generate_v2_novel_need_result.pkl')
neg_res = pd.read_pickle('../gpt_infer_neg_100_result.pkl')

In [42]:
neg_res = [d[1] for d in neg_res]

In [48]:
neg_tmp = neg_res[:8]+neg_res[9:]

In [56]:
neg_process = neg_tmp[:14] + neg_tmp[15:]
print(sum([int(d) for d in neg_process])), print(len([int(d) for d in neg_process]))

36
98


(None, None)

In [57]:
print(sum([int(d[1]) for d in pos_res])), print(len([int(d[1]) for d in pos_res]))

88
91


(None, None)